In [1]:
import os

# Set the directory containing the images
image_folder = "/home/vittorio/Documenti/Upsampling_CFD/results/test_DAT_x2_outdoor_no_umass/visualization/decoded"
# The suffix you want to remove
suffix_to_remove = "_suffix"

# Get all files in the directory
for filename in os.listdir(image_folder):
    # Check if it's a file (not a directory) and if it ends with the suffix you want to remove
    if os.path.isfile(os.path.join(image_folder, filename)):
        # Split the filename into name and extension
        file_name, file_ext = os.path.splitext(filename)
        
        # If the file ends with the suffix, rename it
        if file_name.endswith(suffix_to_remove):
            new_name = file_name[:-len(suffix_to_remove)] + file_ext
            # Rename the file
            os.rename(os.path.join(image_folder, filename), os.path.join(image_folder, new_name))
            print(f"Renamed: {filename} -> {new_name}")

Renamed: XZ_16_33_suffix.png -> XZ_16_33.png
Renamed: XZ_47_39_suffix.png -> XZ_47_39.png
Renamed: XZ_4_36_suffix.png -> XZ_4_36.png
Renamed: XZ_35_40_suffix.png -> XZ_35_40.png
Renamed: XZ_62_39_suffix.png -> XZ_62_39.png
Renamed: XZ_73_35_suffix.png -> XZ_73_35.png
Renamed: XZ_21_40_suffix.png -> XZ_21_40.png
Renamed: XZ_103_35_suffix.png -> XZ_103_35.png
Renamed: XZ_83_35_suffix.png -> XZ_83_35.png
Renamed: XZ_18_32_suffix.png -> XZ_18_32.png
Renamed: XZ_83_40_suffix.png -> XZ_83_40.png
Renamed: XZ_83_38_suffix.png -> XZ_83_38.png
Renamed: XZ_109_32_suffix.png -> XZ_109_32.png
Renamed: XZ_113_39_suffix.png -> XZ_113_39.png
Renamed: XZ_124_38_suffix.png -> XZ_124_38.png
Renamed: XZ_70_34_suffix.png -> XZ_70_34.png
Renamed: XZ_60_31_suffix.png -> XZ_60_31.png
Renamed: XZ_93_31_suffix.png -> XZ_93_31.png
Renamed: XZ_55_35_suffix.png -> XZ_55_35.png
Renamed: XZ_58_39_suffix.png -> XZ_58_39.png
Renamed: XZ_16_37_suffix.png -> XZ_16_37.png
Renamed: XZ_62_38_suffix.png -> XZ_62_38.png
Rena

In [ ]:
import os
import numpy as np
from PIL import Image
import torch
import torch.nn.functional as F
from tqdm import tqdm  # Ensure tqdm is imported for progress bar

def psnr(pred, target, max_val=1.0):
    """
    Compute the Peak Signal-to-Noise Ratio (PSNR) between two images.

    Args:
        pred (torch.Tensor): Predicted image tensor of shape [C, H, W].
        target (torch.Tensor): Target image tensor of shape [C, H, W].
        max_val (float, optional): Maximum possible pixel value. Defaults to 1.0.

    Returns:
        float: PSNR value in decibels (dB).
    """
    mse = F.mse_loss(pred, target)
    if mse == 0:
        return float('inf')  # Perfect match
    psnr_value = 20 * torch.log10(max_val / torch.sqrt(mse))
    return psnr_value.item()

def calculate_psnr_from_folders(pred_folder, target_folder, upscale_factor=1, device='cpu'):
    """
    Calculate the average PSNR between images in two folders.

    Args:
        pred_folder (str): Path to the folder containing predicted/downsampled images.
        target_folder (str): Path to the folder containing target high-resolution images.
        upscale_factor (int, optional): Factor by which to upscale the predicted images.
                                        Defaults to 1 (no upscaling).
        device (torch.device, optional): Device to perform computations on. Defaults to 'cpu'.

    Returns:
        float: Average PSNR value.
    """
    total_psnr = 0.0
    total_images = 0

    # Get sorted lists of image filenames
    pred_images = sorted(os.listdir(pred_folder))
    target_images = sorted(os.listdir(target_folder))

    # Ensure both folders have the same number of images
    #if len(pred_images) != len(target_images):
     #   raise ValueError("Prediction and target folders have different number of images.")

    for pred_img_name, target_img_name in tqdm(zip(pred_images, target_images), total=len(pred_images), desc="Calculating PSNR"):
        pred_path = os.path.join(pred_folder, pred_img_name)
        target_path = os.path.join(target_folder, target_img_name)
        print(f"Processing: {pred_path} and {target_path}")

        # Open images and convert to RGB
        pred_img = Image.open(pred_path).convert('RGB')
        target_img = Image.open(target_path).convert('RGB')

        # Convert images to tensors and normalize
        pred_tensor = torch.from_numpy(np.array(pred_img)).float() / 255.0
        target_tensor = torch.from_numpy(np.array(target_img)).float() / 255.0

        # Ensure the dimensions are [C, H, W]
        pred_tensor = pred_tensor.permute(2, 0, 1).to(device)
        target_tensor = target_tensor.permute(2, 0, 1).to(device)

        # Upscale the predicted image if upscale_factor > 1
        if upscale_factor > 1:
            # Add batch dimension: [1, C, H, W]
            pred_tensor = pred_tensor.unsqueeze(0)

            # Perform upscaling
            pred_tensor = F.interpolate(pred_tensor, scale_factor=upscale_factor, mode='bicubic', align_corners=False)

            # Remove batch dimension: [C, H, W]
            pred_tensor = pred_tensor.squeeze(0)

        # Compute PSNR
        psnr_value = psnr(pred_tensor, target_tensor, max_val=1.0)
        total_psnr += psnr_value
        total_images += 1

    # Compute average PSNR
    avg_psnr = total_psnr / total_images
    return avg_psnr

# Define folder paths
pred_folder_resShift = "/home/vittorio/Documenti/Upsampling_CFD/results/test_DAT_x2_outdoor_no_umass/visualization/decoded"
pred_folder_bicubic_downsampled = '/home/vittorio/Scrivania/ETH/Upsampling/Upsampling_CFD/datasets/Split_Outdoor_Flow/test/low_res'
target_folder_resShift = '/home/vittorio/Scrivania/ETH/Upsampling/ResShift_4_scale/data/Scalar_Images'
# The target_folder_resShift is the same for bicubic as it's the high-resolution images

# Define device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Calculate PSNR for the model's output (ResShift)
avg_psnr_resShift = calculate_psnr_from_folders(
    pred_folder=pred_folder_resShift,
    target_folder=target_folder_resShift,
    upscale_factor=1,  # No upscaling needed for model outputs
    device=device
)

# Calculate PSNR for Bicubic upscaled images
# Since pred_folder_bicubic_downsampled contains downsampled images, we upscale them by a factor of 4
avg_psnr_bicubic = calculate_psnr_from_folders(
    pred_folder=pred_folder_bicubic_downsampled,
    target_folder=target_folder_resShift,
    upscale_factor=2,  # Upscale by a factor of 4
    device=device
)

# Print the results
print(f"Validation PSNR - Model Output (ResShift): {avg_psnr_resShift:.2f} dB")
print(f"Validation PSNR - Bicubic Upscaling: {avg_psnr_bicubic:.2f} dB")

Calculating PSNR:   0%|          | 0/252 [00:00<?, ?it/s]<ipython-input-3-68437fa6d575>:20: UserWarning: Using a target size (torch.Size([3, 24, 24])) that is different to the input size (torch.Size([3, 48, 48])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  mse = F.mse_loss(pred, target)
Calculating PSNR:   1%|          | 2/252 [00:00<00:02, 110.40it/s]

Processing: /home/vittorio/Documenti/Upsampling_CFD/results/test_DAT_x2_outdoor_no_umass/visualization/decoded/XZ_100_31.png and /home/vittorio/Scrivania/ETH/Upsampling/ResShift_4_scale/data/Scalar_Images/XZ_0_30.png
Processing: /home/vittorio/Documenti/Upsampling_CFD/results/test_DAT_x2_outdoor_no_umass/visualization/decoded/XZ_101_34.png and /home/vittorio/Scrivania/ETH/Upsampling/ResShift_4_scale/data/Scalar_Images/XZ_0_30_1.png
Processing: /home/vittorio/Documenti/Upsampling_CFD/results/test_DAT_x2_outdoor_no_umass/visualization/decoded/XZ_101_40.png and /home/vittorio/Scrivania/ETH/Upsampling/ResShift_4_scale/data/Scalar_Images/XZ_0_30_2.png


RuntimeError: The size of tensor a (48) must match the size of tensor b (24) at non-singleton dimension 2

: 

In [3]:
import os
import numpy as np
from PIL import Image
import torch
import torch.nn.functional as F
from tqdm import tqdm  # Import tqdm for progress bars

def ssim(pred, target, C1=0.01**2, C2=0.03**2):
    mu_x = F.conv2d(pred, weight=torch.ones([pred.shape[1], 1, 11, 11]).to(pred.device) / 121, padding=5, groups=pred.shape[1])
    mu_y = F.conv2d(target, weight=torch.ones([target.shape[1], 1, 11, 11]).to(target.device) / 121, padding=5, groups=target.shape[1])

    sigma_x = F.conv2d(pred * pred, weight=torch.ones([pred.shape[1], 1, 11, 11]).to(pred.device) / 121, padding=5, groups=pred.shape[1]) - mu_x ** 2
    sigma_y = F.conv2d(target * target, weight=torch.ones([target.shape[1], 1, 11, 11]).to(target.device) / 121, padding=5, groups=target.shape[1]) - mu_y ** 2
    sigma_xy = F.conv2d(pred * target, weight=torch.ones([pred.shape[1], 1, 11, 11]).to(pred.device) / 121, padding=5, groups=pred.shape[1]) - mu_x * mu_y

    ssim_n = (2 * mu_x * mu_y + C1) * (2 * sigma_xy + C2)
    ssim_d = (mu_x ** 2 + mu_y ** 2 + C1) * (sigma_x + sigma_y + C2)

    ssim = ssim_n / ssim_d
    return ssim.mean()

def calculate_ssim_from_folders(pred_folder, target_folder, upscale_factor=1, device='cpu'):
    """
    Calculate the average SSIM between images in two folders.

    Args:
        pred_folder (str): Path to the folder containing predicted/downsampled images.
        target_folder (str): Path to the folder containing target high-resolution images.
        upscale_factor (int, optional): Factor by which to upscale the predicted images.
                                        Defaults to 1 (no upscaling).
        device (torch.device, optional): Device to perform computations on. Defaults to 'cpu'.

    Returns:
        float: Average SSIM value.
    """
    total_ssim = 0.0
    total_images = 0

    # Get sorted lists of image filenames
    pred_images = sorted(os.listdir(pred_folder))
    target_images = sorted(os.listdir(target_folder))

    # Ensure both folders have the same number of images
    if len(pred_images) != len(target_images):
        raise ValueError("Prediction and target folders have different number of images.")

    for pred_img_name, target_img_name in tqdm(zip(pred_images, target_images), total=len(pred_images), desc="Calculating SSIM"):
        pred_path = os.path.join(pred_folder, pred_img_name)
        target_path = os.path.join(target_folder, target_img_name)

        # Open images and convert to RGB
        pred_img = Image.open(pred_path).convert('RGB')
        target_img = Image.open(target_path).convert('RGB')

        # Convert images to tensors and normalize
        pred_tensor = torch.from_numpy(np.array(pred_img)).float() / 255.0
        target_tensor = torch.from_numpy(np.array(target_img)).float() / 255.0

        # Ensure the dimensions are [C, H, W]
        pred_tensor = pred_tensor.permute(2, 0, 1).to(device)  # Shape: [C, H, W]
        target_tensor = target_tensor.permute(2, 0, 1).to(device)  # Shape: [C, H, W]

        # Upscale the predicted image if upscale_factor > 1 using PyTorch's interpolate
        if upscale_factor > 1:
            # Add batch dimension: [1, C, H, W]
            pred_tensor = pred_tensor.unsqueeze(0)

            # Calculate new spatial dimensions based on upscale_factor
            # Alternatively, you can use scale_factor=upscale_factor in interpolate
            # but here we ensure it matches the target dimensions
            target_height, target_width = target_tensor.shape[1], target_tensor.shape[2]
            pred_tensor = F.interpolate(pred_tensor, size=(target_height, target_width), mode='bicubic', align_corners=False)

            # Remove batch dimension: [C, H, W]
            pred_tensor = pred_tensor.squeeze(0)

        # Add batch dimension for SSIM function: [1, C, H, W]
        pred_tensor = pred_tensor.unsqueeze(0)
        target_tensor = target_tensor.unsqueeze(0)

        # Compute SSIM
        with torch.no_grad():
            ssim_value = ssim(pred_tensor, target_tensor).item()
        total_ssim += ssim_value
        total_images += 1

    # Compute average SSIM
    avg_ssim = total_ssim / total_images
    return avg_ssim

# Define folder paths
# Define folder paths
pred_folder_resShift = "/home/vittorio/Scrivania/ETH/Upsampling/Upsampling_CFD/results/swinir_classical_sr_x2"
pred_folder_bicubic_downsampled = '/home/vittorio/Scrivania/ETH/Upsampling/Upsampling_CFD/datasets/Split_Outdoor_Flow/test/low_res'
target_folder_resShift = '/home/vittorio/Scrivania/ETH/Upsampling/Upsampling_CFD/datasets/Split_Outdoor_Flow/test/high_res'
# Define device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Calculate SSIM for the model's output (ResShift)
avg_ssim_resShift = calculate_ssim_from_folders(
    pred_folder=pred_folder_resShift,
    target_folder=target_folder_resShift,
    upscale_factor=1,  # No upscaling needed for model outputs
    device=device
)

# Calculate SSIM for Bicubic upscaled images
avg_ssim_bicubic = calculate_ssim_from_folders(
    pred_folder=pred_folder_bicubic_downsampled,
    target_folder=target_folder_resShift,
    upscale_factor=2,  # Upscale by a factor of 4
    device=device
)

# Print the results
print(f"\nValidation SSIM - Model Output (ResShift): {avg_ssim_resShift:.4f}")
print(f"Validation SSIM - Bicubic Upscaling: {avg_ssim_bicubic:.4f}")

Calculating SSIM: 100%|██████████| 193/193 [00:00<00:00, 254.74it/s]


Validation SSIM - Model Output (ResShift): 0.9793
Validation SSIM - Bicubic Upscaling: 0.8704
